In [11]:
##Imports and setup for Jupyter

import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from bs4 import BeautifulSoup
from IPython import display
import re

##function should allow to run in jupyter-- use show_app(#nameofapp) 

def show_app(app,  # type: dash.Dash
             port=9999,
             width=700,
             height=350,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    """
    Run the application inside a Jupyter notebook and show an iframe with it
    :param app:
    :param port:
    :param width:
    :param height:
    :param offline:
    :return:
    """
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,  # needs to be false in Jupyter
                          port=port,
                          **dash_flask_kwargs)



## General Dash Notes

anything that starts with html. ... is generating pure HTML, anything that comes from the dcc. library is using things like Javascript, CSS and HTML

can customize the CSS styles of the app---app.css.append_css()

have to stop the kernal and restart each time you'd like to 'retest' the app--- guessing that it stays running so that you can make interactive changes to it.

style property is best to supply a dictionary to (colors example



In [12]:
##Importing the final report

##can I call something from another jupyter notebook? Seems like we might need to either have a massive setup beforehand or
##save the other file as a .py script and then import like that

report=pd.read_csv('connect_report.csv')

report['Created Date']=pd.to_datetime(report['Created Date'])
report=report[pd.notnull(report['Created Date'])]
report=report[pd.notnull(report['Body'])]

#Removing HTML tags and codes
report['Body'] = report['Body'].apply(lambda x:BeautifulSoup(x))
report['Body'] = report['Body'].apply(lambda x:x.get_text())

#Removing tags/mentions
report['Body'] = report['Body'].apply(lambda x:re.sub('{@[\w\d]*}', '',x))

#Removing weird unicode characters
report['Body'] = report['Body'].apply(lambda x: x.encode('ascii', 'ignore'))


final_report=pd.DataFrame

#Use this function to create the final report
# Directions:
# 1. Set Group to a list of group names, a single name of a group, or All to use all groups
# 2. Set Type to a specific Feed Item Type or All
# 3. Set Start Date or leave blank(defaults to: )
# 4. Set End Date or leave blank(defaults to: )

def report_generator(Group,Type,Date_start=pd.to_datetime("2017-01-01"),Date_finish=pd.to_datetime('today')):
    if type(Group) == list:
        final_report=report.loc[report['Name'].isin(Group)] 
    elif (Group=='All'):
        final_report=report
    else:
        final_report=report.loc[report['Name']==Group]
                                
    if (Type=='All'):
        final_report=final_report
    else:
        final_report=final_report.loc[final_report['Feed Item Type']==Type]
    
    if(Date_start=="2017-01-01" and Date_finish== pd.to_datetime('today')):
        final_report=final_report
    else:
        Date_start=pd.to_datetime(Date_start, utc=False)
        Date_finish=pd.to_datetime(Date_finish, utc=False)
        #alt approach-- not working but not sure why-- it worked it worked it worked! had to change the data type to all be annoying ass datatime64... down the line that might be a pain but for now it's dope
        final_report=final_report[(final_report['Created Date'] > Date_start) & (final_report['Created Date']< Date_finish)]   
    return(final_report)

final_report=report_generator(['CPC+ All','NLT Internal Users'],'All',"2017-02-01","2017-03-01")



In [20]:
##Setting up table generator example
def generate_table(dataframe, maxrows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), maxrows))]
    )


In [22]:
##Setting up a test environment



app=dash.Dash()

colors={
    'background': '#111111',
    'text': '#7FDBFF'
}

app.layout=html.Div(children=
    [html.H1(children='This is the title pane',
             style={
                 'textAlign': 'center', 'color':colors['text']}   ),
    html.Div(children='''
                        This is the first body div''', 
            style={
                'textAlign':'center',
                'color':colors['text']
            }),
    dcc.Graph(
        id='test_graph',
        figure={
            'data': [
                {'x': [1, 2, 3], 'y': [4, 1, 2], 'type': 'bar', 'name': 'SF'},
                {'x': [1, 2, 3], 'y': [2, 4, 5], 'type': 'bar', 'name': u'Montréal'},
            ],
            'layout':{
                'title': 'First Data Viz',
                'plot_bgcolor':colors['background'],
                'paper_bgcolor':colors['background'],
                'font':{
                    'color':colors['text']
                }
                
            }
        }
    ),
    html.H4(children='This is the Connect report for the specified time'),
     generate_table(final_report,10)
])


show_app(app)

 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Nov/2017 11:54:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2017 11:54:48] "GET /_dash-component-suites/dash_core_components/rc-slider@6.1.2.css?v=0.14.0 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2017 11:54:48] "GET /_dash-component-suites/dash_core_components/react-select@1.0.0-rc.3.min.css?v=0.14.0 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2017 11:54:48] "GET /_dash-component-suites/dash_core_components/react-virtualized@9.9.0.css?v=0.14.0 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2017 11:54:48] "GET /_dash-component-suites/dash_core_components/react-virtualized-select@3.1.0.css?v=0.14.0 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2017 11:54:48] "GET /_dash-component-suites/dash_core_components/react-dates@12.3.0.css?v=0.14.0 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2017 11:54:48] "GET /_dash-component-suites/dash_renderer/react@15.4.2.min.js?v=0.11.1 HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2017 11:54:48] "GET /_dash-componen